In [ ]:
## load news data from database into dataframe anc cluster with https://towardsdatascience.com/all-the-news-17fa34b52b9d

In [24]:
import sys
for p in sys.path:
    print(p)

/workspaces/data-app/app/database
database
.
/workspaces/data-app/app/news_clustering
/usr/local/lib/python311.zip
/usr/local/lib/python3.11
/usr/local/lib/python3.11/lib-dynload

/home/vscode/.local/lib/python3.11/site-packages
/usr/local/lib/python3.11/site-packages


In [23]:
sys.path.insert(0,'/workspaces/data-app/app/database')

In [26]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio
from database import Database

In [54]:
# preprocessing
import re
import sys
import nltk
#nltk.download('stopwords')
# nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
from collections import Counter
import time

# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
database = Database()
database.open_connection()

In [32]:
## get articles table into datframe
sql = "SELECT * FROM articles;"
articles = sqlio.read_sql_query(sql, database.connection)
database.close_connection()

/tmp/ipykernel_1052/2993504995.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  articles = sqlio.read_sql_query(sql, database.connection)


In [33]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   urlid              49 non-null     object        
 1   headline           45 non-null     object        
 2   content            49 non-null     object        
 3   authors            49 non-null     object        
 4   uploadtimestamp    38 non-null     datetime64[ns]
 5   imageurl           49 non-null     object        
 6   imagedescription   49 non-null     object        
 7   scrapingtimestamp  49 non-null     datetime64[ns]
 8   source             49 non-null     object        
 9   topic              49 non-null     object        
dtypes: datetime64[ns](2), object(8)
memory usage: 4.0+ KB


## Plan:
### 1) stemming
### 2) create word matrix
### 3) TFIDF
### 4) Clustering
### 5) majority vote of category within clusters to be used as cluster name

In [45]:
type(articles)

pandas.core.frame.DataFrame

In [41]:
# Data Analytics preprocessing

# Option A

# Preprocessing
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [word for word in words if word not in stopwords.words("english")]  # Remove stopwords
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]  # Stem words
    return ' '.join(words)


In [47]:
# Option B

# Preprocessing
def preprocess_lemmatizer(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [word for word in words if word not in stopwords.words("english")]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]  # Stem words
    return ' '.join(words)

In [52]:
preprocessed_articles = articles.content.apply(lambda x: preprocess(x))
print(preprocessed_articles[0])

post hostag famili urg return negoti tabl rel hostag taken israel hama call isra govern return negoti tabl price promis us price would return hostag daniel lifshitz whose grandpar yochev ode lifshitz taken hostag among famili member speak news confer morn call cabinet meet order forget feel forgot time everyth except famili ignor indiffer lack attent us disgrac said reuter handout pictur copyright reuter handout pictur yochev lifshitz husband ode yochev freed ode still captiv imag caption yochev lifshitz husband ode yochev freed ode still captiv haim yitzhak brother avinatan kidnap supernova festiv told report famili dont think interven tactic manag war want answer promis us would meet us didnt happen dont know isra prime minist offic respond grow frustrat famili meet famili hostag war cabinet alreadi schedul yesterday wednesday famili request possibl reschedul earlier examin articl share tool share view share option share post copi link read link


In [51]:
preprocessed_lemmatized_articles = articles.content.apply(lambda x: preprocess_lemmatizer(x))
print(preprocessed_lemmatized_articles[0])

posted hostage family urge return negotiating table relative hostage taken israel hamas calling israeli government return negotiating table price promised u price would return hostage daniel lifshitz whose grandparent yocheved oded lifshitz taken hostage among family member speaking news conference morning call cabinet meet order forget feel forgot time everything except family ignoring indifference lack attention u disgrace said reuters handout picture copyright reuters handout picture yocheved lifshitz husband oded yocheved freed oded still captivity image caption yocheved lifshitz husband oded yocheved freed oded still captivity haim yitzhak brother avinatan kidnapped supernova festival told reporter family dont think intervening tactical management war want answer promised u would meet u didnt happen dont know israeli prime minister office responded growing frustration family meeting family hostage war cabinet already scheduled yesterday wednesday family request possibility resched

## ToDo: first add cnn articles to DB and rerun code
### second choose one way of preprocessing

In [57]:
# Transform into numerical data and apply cosine similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_articles)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.01918833 0.04558346 ... 0.         0.01838554 0.01574456]
 [0.01918833 1.         0.05203623 ... 0.00818932 0.02765311 0.02653302]
 [0.04558346 0.05203623 1.         ... 0.         0.04630605 0.05853878]
 ...
 [0.         0.00818932 0.         ... 1.         0.02684336 0.        ]
 [0.01838554 0.02765311 0.04630605 ... 0.02684336 1.         0.11560519]
 [0.01574456 0.02653302 0.05853878 ... 0.         0.11560519 1.        ]]


## from here down: to be deleted!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [34]:
## 1) stemming
stemmer = PorterStemmer()
##tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
progress = 0 #for keeping track of where the function is

def stemm(x):
   end = time.time()
   dirty = word_tokenize(x)
   tokens = []
   for word in dirty:
      if word.strip('.') == '': #this deals with the bug
         pass
      elif re.search(r'\d{1,}', word): #getting rid of digits
         pass
      else:
         tokens.append(word.strip('.'))
   global start
   global progress
   tokens = pos_tag(tokens) #
   progress += 1
   stems = ' '.join(stemmer.stem(key.lower()) for key, value in  tokens if value != 'NNP') #getting rid of proper nouns
   end = time.time()
   sys.stdout.write('\r {} percent, {} position, {} per second '.format(str(float(progress / len(articles))), str(progress), (1 / (end - start)))) #lets us see how much time is left 
   start = time.time()
   return stems

In [55]:
# start = time.time()
# articles['stems'] = articles.content.apply(lambda x: stemm(x))